In [1]:
# requirements
import yfinance as yf
import pandas as pd
import numpy as np

In [2]:
# example tickers
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'FB']

# download historical data
data = yf.download(tickers, start='2020-01-01', end='2023-01-01')['Adj Close']

# calculate daily returns
returns = data.pct_change()

# Calculate additional technical indicators
data['SMA_20'] = data.rolling(window=20).mean()
data['SMA_50'] = data.rolling(window=50).mean()
data['EMA_20'] = data.ewm(span=20, adjust=False).mean()
data['Momentum'] = returns.rolling(window=20).mean()

# Merge all indicators into a single DataFrame
indicators = pd.concat([data, returns.rename('Returns')], axis=1)
indicators.tail()


[*********************100%%**********************]  5 of 5 completed

1 Failed download:
['FB']: Exception('%ticker%: No timezone found, symbol may be delisted')
/var/folders/y8/fcjyhk1d0j3f5lg1dm4m054r0000gn/T/ipykernel_12268/1130746924.py:8: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = data.pct_change()


ValueError: Cannot set a DataFrame with multiple columns to the single column SMA_20